In [22]:
import re
import spacy
from gensim import corpora # necessary for building dictionary and bag of words representation
from gensim import models # necessary for Phrases, LDA model and TFIDF

In [23]:
nlp = spacy.load("en")

In [24]:
corpus = "snf_html_corpus_simple.txt"

raw_corpus = []

with open(corpus, "r", encoding="utf8") as inf:    
    doc = ""
    for line in inf:
        if not line.startswith("\n"): # we are inside a document
            line = re.sub("\xa0", " ", line.rstrip())
            doc += line.lower() # lower and add the line to the document string
        else: # we are at a document boundary
            if len(doc) > 380: # ignore shorter documents
                raw_corpus.append(doc) # should end up with 110 articles in list of docs 
            doc = ""

# inspect the raw_corpus:
# raw_corpus

In [25]:
# testing on one doc:

# pro_doc = nlp(docs[0])
# print(pro_doc)

# Inspect entities found
# pro_doc.ents

# for token in pro_doc[:100]:
#     print(token.text, token.lemma_, token.pos_, token.tag_,
#           token.dep_, token.shape_, token.is_alpha, token.is_stop)

In [26]:
# add stopwords
noisy_words = ["research", "say", "says", "said", "in", "not", "this", "researcher"]

for w in noisy_words:
    nlp.vocab[w].is_stop = True

In [27]:
# test = nlp("The research says that this man is not a researcher, but he is.".lower())
# for w in test:
#     if not w.is_stop:
#         print(w)

In [28]:
corpus, article = [], []

for doc in raw_corpus:
    doc = nlp(doc)
    for w in doc:
        # if it's not a stop word or punctuation mark, add it to our article!
        if w.text != " " and not w.is_stop and not w.is_punct and not w.like_num:
            # we add the lematized version of the word
            lemma = w.lemma_.strip("’") # removes things like "science’"
            if lemma:
                article.append(lemma)
    corpus.append(article)
    article = []

# inspect corpus
corpus

[['observer',
  'consider',
  'overcome',
  'issue',
  'require',
  'new',
  'paradigm',
  'open',
  'science',
  'idea',
  'break',
  'shackle',
  'fetter',
  'individual',
  'element',
  'scientific',
  'production',
  'process',
  'formation',
  'hypothesis',
  'publication',
  'result',
  'infographic',
  'element',
  'open',
  'science',
  'p.',
  'watchword',
  'sharing',
  'inclusion',
  'collaboration',
  'decentralisation',
  'transparency',
  'fully',
  'open',
  'work',
  'useful',
  'scientist',
  'industry',
  'member',
  'civil',
  'society',
  'computer',
  'program',
  'able',
  'draw',
  'new',
  'conclusion',
  'old',
  'result',
  'pillar',
  'open',
  'access',
  'aim',
  'ensure',
  'scientific',
  'publication',
  'accessible',
  'free',
  'charge',
  'people',
  'little',
  'impatient',
  'clearly',
  'right',
  'track',
  'daniël',
  'laken',
  'psychology',
  'practitioner',
  'open',
  'science',
  'eindhoven',
  'university',
  'technology',
  'accord',
  'eu

In [29]:
trained_phrases = models.phrases.Phrases(corpus) # train a phrase model
phraser = models.phrases.Phraser(trained_phrases) # initialise the trained phraser model
corpus_phrased = [phraser[text] for text in corpus] # apply phraser model to each text in corpus
# inspect the first text in the corpus after applying the phrase model
# corpus_phrased[3]

In [30]:
dictionary = corpora.Dictionary(corpus_phrased)
mapped_corpus = [dictionary.doc2bow(doc) for doc in corpus_phrased]
# inspect mapped_corpus
# corpus[1][0:10]

In [31]:
def print_topics_gensim(topic_model, total_topics=1, weight_threshold=0.0001, num_terms=None):
    """Adapted pretty print topic model results from Sakar (2016) Text Analytics With Python."""
    for index in range(total_topics):
        topic = topic_model.show_topic(index)
        topic = [(word, round(wt,4)) for word, wt in topic if abs(wt) >= weight_threshold]
        print('Topic #'+str(index+1))
        print(topic[:num_terms] if num_terms else topic)
        print()

def train_lda_model_gensim(corpus, total_topics=2):
    """Adapted pretty print topic model results from Sakar (2016) Text Analytics With Python."""
    dictionary = corpora.Dictionary(corpus)
    mapped_corpus = [dictionary.doc2bow(text) for text in corpus]
    tfidf = models.TfidfModel(mapped_corpus)
    corpus_tfidf = tfidf[mapped_corpus]
    lda = models.LdaModel(corpus_tfidf, id2word=dictionary, iterations=1000, num_topics=total_topics)
    return lda

In [32]:
def run_model_for_X_topics(X=2):
    lda_gensim = train_lda_model_gensim(corpus_phrased, total_topics=X)
    print_topics_gensim(topic_model=lda_gensim, total_topics=X, num_terms=10)

In [33]:
run_model_for_X_topics(10)

Topic #1
[('selenium', 0.0011), ('disgust', 0.0008), ('ghernaouti', 0.0008), ('markwalder', 0.0007), ('robot', 0.0007), ('hartmann', 0.0007), ('sentence', 0.0007), ('public', 0.0007), ('criminal', 0.0006), ('justice_system', 0.0006)]

Topic #2
[('sign_language', 0.0011), ('memristor', 0.0009), ('caffeine', 0.0009), ('underground', 0.0008), ('be', 0.0007), ('chafer', 0.0007), ('energy', 0.0007), ('general_consent', 0.0007), ('subsoil', 0.0007), ('legislation', 0.0007)]

Topic #3
[('elephant', 0.0009), ('gravitational', 0.0008), ('cement', 0.0008), ('küffer', 0.0008), ('rickli', 0.0007), ('lösch', 0.0007), ('age', 0.0007), ('biodiversity', 0.0006), ('video_game', 0.0006), ('pelican', 0.0006)]

Topic #4
[('vayena', 0.001), ('patient', 0.001), ('papetti', 0.0008), ('war', 0.0008), ('mountain', 0.0007), ('medical', 0.0007), ('populist', 0.0007), ('soil', 0.0007), ('robot', 0.0007), ('datum', 0.0007)]

Topic #5
[('language', 0.0011), ('game', 0.0009), ('english', 0.0007), ('dyslexia', 0.0007